## DecisionTreeClassifier

In [ ]:
# pip install imblearn
# pip install pydotplus

In [ ]:
# Importing Packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,MinMaxScaler
from imblearn.over_sampling import SMOTE
from sklearn.datasets import load_iris
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,plot_tree,DecisionTreeRegressor
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import pydotplus

## DecisionTree1

In [ ]:
# Importing Dataset

app_df = pd.read_csv("application_record.csv")
cr_df = pd.read_csv("credit_record.csv")

In [ ]:
# Display Top 5 Rows Of Application Record

app_df.head()

In [ ]:
# Display Top 5 Rows Of Credit Record

cr_df.head()

In [ ]:
# Check Shape of Application Record

app_df.shape

In [ ]:
# Check Shape of Credit Record

cr_df.shape

In [ ]:
# Preparing The Target Variable
# 1. For customers having due for multiple months,the maximum number of months due is taken.
# 2. Any ID with greater than equal to 1 months due is marked as Class'1'

cr_df = cr_df.sort_values(['ID','MONTHS_BALANCE'],ascending = False)
cr_df = cr_df.groupby("ID").agg(max).reset_index()

In [ ]:
# Status : 0 => 1-29 days overdue;1 => 30-59 days overdue;2 => 60-89 days overdue;3 => 90-119 days overdue;4 => 120-149 
# days overdue;5 => overdue or bad debts,write offs for more than 150 days;C => paid off that month;X => No loan for the 
# month.

cr_df["STATUS"].replace({"C":0,"X":0},inplace=True)
cr_df["STATUS"] = cr_df["STATUS"].astype("int")
cr_df["STATUS"] = cr_df["STATUS"].apply(lambda x:1 if x>0 else 0)

In [ ]:
# Merging Both Application And Credit Status Datasets

df = app_df.join(cr_df.set_index("ID"),on="ID",how="inner")

In [ ]:
# Information About Dataset

df.info()

In [ ]:
# Get Null Value From The Dataset

df.isnull().sum()

In [ ]:
# Display Top 5 Rows Of Dataset

df.head()

In [ ]:
# Different Type Of Features Based On Data Type

binary_features = ["CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","FLAG_WORK_PHONE","FLAG_EMAIL"]
continuous_features = ["CNT_CHILDREN","AMT_INCOME_TOTAL","DAYS_BIRTH","DAYS_EMPLOYED","CNT_FAM_MEMBERS"]
cat_features = ["NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE"]

In [ ]:
# Dropping Features
# 1. OCCUPATION_TYPE - Lot Of Missing Data (11323 Data Points)
# 2. FLAG_MOBIL - All Datapoints Have The Flag Set To 1

df.drop(["MONTHS_BALANCE","OCCUPATION_TYPE","FLAG_MOBIL"],inplace=True,axis=1)

In [ ]:
# Ordering Ordinal Values And Encoding : We Have Determined The Order Of Income Type (A Qualitative Call) And Created The
# Encoding Of The Variable "NAME_INCOME_TYPE"

encoder = OrdinalEncoder(categories=[["Student","Pensioner","Working","Commercial associate","State servant"]])
df["NAME_INCOME_TYPE"] = encoder.fit_transform(df["NAME_INCOME_TYPE"].values.reshape(-1,1))

In [ ]:
# Encoding Ordinal Columns Without Ordering

encoder2 = LabelEncoder()
for i in cat_features[1:]:
    df[i] = encoder2.fit_transform(df[i].values.reshape(-1,1))

In [ ]:
# For Binary Features We Had Used Pandas Dummies For One Hot Encoding

df = pd.get_dummies(df,columns=binary_features,drop_first=True)

In [ ]:
# Plotting Different Variables To Get An Understanding Of Outlier Present

fig,ax = plt.subplots(nrows=2,ncols=3,figsize=(14,6))
sns.scatterplot(x="ID",y="CNT_CHILDREN",data=df,ax=ax[0][0],color="orange")
sns.scatterplot(x="ID",y="AMT_INCOME_TOTAL",data=df,ax=ax[0][1],color="orange")
sns.scatterplot(x="ID",y="DAYS_BIRTH",data=df,ax=ax[0][2])
sns.scatterplot(x="ID",y="DAYS_EMPLOYED",data=df,ax=ax[1][0])
sns.scatterplot(x="ID",y="CNT_FAM_MEMBERS",data=df,ax=ax[1][1],color="orange")

In [ ]:
# Removing Outlier Using IQR Technique

q3 = df["AMT_INCOME_TOTAL"].quantile(.75)
q1 = df["AMT_INCOME_TOTAL"].quantile(.25)
IQR = q3 - q1
lower_range = q1 - 1.5 * IQR
upper_range = q3 + 1.5 * IQR
df = df[(df["AMT_INCOME_TOTAL"] > lower_range) & (df["AMT_INCOME_TOTAL"] < upper_range)]
df = df[df["CNT_CHILDREN"] < 8]
df = df[df["CNT_FAM_MEMBERS"] < 8]

In [ ]:
# Plotting Post Removal Of Outlier

fig,ax = plt.subplots(nrows=2,ncols=3,figsize=(14,6))
sns.scatterplot(x="ID",y="CNT_CHILDREN",data=df,ax=ax[0][0],color="orange")
sns.scatterplot(x="ID",y="AMT_INCOME_TOTAL",data=df,ax=ax[0][1],color="orange")
sns.scatterplot(x="ID",y="DAYS_BIRTH",data=df,ax=ax[0][2])
sns.scatterplot(x="ID",y="DAYS_EMPLOYED",data=df,ax=ax[1][0])
sns.scatterplot(x="ID",y="CNT_FAM_MEMBERS",data=df,ax=ax[1][1],color="orange")

In [ ]:
# Train Test Split

Y = df["STATUS"]
X = df.drop("STATUS",axis=1)

In [ ]:
# Split The Data Into Training And Testing

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)
mms = MinMaxScaler()
X_scaled = pd.DataFrame(mms.fit_transform(X_train),columns=X_train.columns)
sm = SMOTE(random_state=42)
X_train_res,Y_train_res = sm.fit_resample(X_scaled,Y_train)

In [ ]:
# Checking The Shape Of New Dataset

unique,counts = np.unique(Y_train_res,return_counts=True)
np.asarray((unique,counts))

In [ ]:
# Create DecisionTreeClassifier

classifier = DecisionTreeClassifier()
model = classifier.fit(X_train_res,Y_train_res)
X_test_scaled = mms.transform(X_test)
prediction = model.predict(X_test_scaled)

In [ ]:
# Check Model Performance

print("Accuracy Score : {:.5}".format(accuracy_score(Y_test,prediction)))
print(pd.DataFrame(confusion_matrix(Y_test,prediction)))

In [ ]:
# Display Classification Report

print(classification_report(Y_test,prediction))

In [ ]:
# Display Feature Importance

feats = {}
for feature,importance in zip(X_train.columns,model.feature_importances_):
    feats[feature] = importance
importances = pd.DataFrame().from_dict(feats,orient="index").rename(columns={0:"Gini-Importance"})
importances.sort_values(by="Gini-Importance").plot(kind="bar",rot=90)

## DecisionTree2

In [ ]:
# Importing Dataset

iris = load_iris()
X = iris.data[:,:2]  # Take only first two feature
Y = iris.target

In [ ]:
# Split Data Into Training And Testing Data

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
# Build Model

clf = DecisionTreeClassifier(max_depth=3,min_samples_split=40)
clf.fit(X_train,Y_train)

In [ ]:
# Predict The Output

Y_pred = clf.predict(X_test)
print(f"Accuracy : {accuracy_score(Y_test,Y_pred)}")

In [ ]:
# Plot The Decision Tree

plt.figure(figsize=(12,8))
plot_tree(clf,filled=True,feature_names = iris.feature_names[:2],class_names=iris.target_names)
plt.show()

# Plot The Decision Boundary

plt.figure(figsize=(8,6))
x_min,x_max = X[:,0].min() - 1,X[:,0].max() + 1
y_min,y_max = X[:,1].min() - 1,X[:,1].max() + 1
xx,yy = np.meshgrid(np.arange(x_min,x_max,0.01),np.arange(y_min,y_max,0.01))

Z = clf.predict(np.c_[xx.ravel(),yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx,yy,Z,alpha=0.8)
plt.scatter(X[:,0],X[:,1],c=Y,edgecolor='k')
plt.xlabel("Sepal Length")
plt.ylabel("Sepal Width")
plt.show()

In [ ]:
# Plot Decision Tree In Text Format

r = export_text(clf,feature_names=["sepal_length","sepal_width"])
print(r)

In [ ]:
# Get Feature Importances

clf.feature_importances_

## DecisionTree3

In [ ]:
# Generate Dataset

np.random.seed(0)

X = np.sort(5 * np.random.rand(80,1),axis=0)
Y = np.sin(X).ravel()
Y[::5] += 3*(0.5 - np.random.rand(16)) # Add Noise To The Data

In [ ]:
# Split The Data Into Training And Testing Data

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state = 42)

In [ ]:
# Build DecisionTreeRegressor

regressor = DecisionTreeRegressor()
regressor.fit(X_train,Y_train)

In [ ]:
# Plot Output

X_grid = np.arange(min(X),max(X),0.01)[:,np.newaxis]
Y_grid = regressor.predict(X_grid)

plt.scatter(X_train,Y_train,color="red",label="data")
plt.plot(X_grid,Y_grid,color="blue",label="prediction")
plt.title("Decision Tree Regressor")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()

In [ ]:
# Plot Decision Tree

plt.figure(figsize=(12,8))
plot_tree(regressor,filled=True)
plt.show()